# Anime Ratings Data 

We didn't really get the databowl data/ wanted to start with something I wanted to try for some time now. <br>
I wanted to predict whether a current figurine(say American comic kind/jpop culture kind,... basically merchandise will skyrocket in price or not) <br>
I wanted to find data on different collectables instead but such a dataset does not exist/ probably have to scrape or use an outdated API(?)
We want to predict ratings for anime

## Exploring time 


Importing stuff oop

In [10]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import joblib
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier,forest 
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn import metrics

###### *To note*
I made a folder in my data folder, databowl, i was lazy to change it <br>
this PATH is assuming my work directory is the ml1 folder in courses <br>
Ok wait ah its not

In [15]:
PATH = "databowl" 

In [16]:
!ls {PATH}

'ls' is not recognized as an internal or external command,
operable program or batch file.


prediction variable: rating. Can drop anime ID. <br>
to determine proportion to split for  test/training 

In [343]:
df_raw = pd.read_csv(f'{PATH}/datasets_571_1094_anime.csv', low_memory=False)

### Saving for another time 
Well the file is kind of big and i want to sleep <br>
i will save my pickle format file in databowl folder so github wont read it :D 

In [18]:
import pickle

In [30]:
??open

In [20]:
import os
print(os.getcwd())

C:\Users\miche\dataBowl


In [21]:
with open("databowl\\peepee", 'wb') as f:
    pickle.dump(df_raw,f)

### Note 
you can run this code instead of reading in the csv - think i included the tmp file 

In [22]:
df_raw = pickle.load(open("databowl\\peepee", 'rb'))

In [23]:
df_raw.episodes = pd.to_numeric(df_raw.episodes, errors = 'coerce')

In [36]:
subset_raw = df_raw.sample(n=2000)

## put the type into numerical values, Movie, Music, OVA, ONA, TV, Special, type, and Blanks (total 7+1)  # i try to do with 'type' first instead of 'genre'.
#### will need to treat the missing values before applying label encoder
#####https://www.youtube.com/watch?v=ZS0hzcA5w9I


In [396]:
df_raw = pd.read_csv(f'{PATH}/datasets_571_1094_anime.csv', low_memory=False)

In [355]:
df_raw['type'].value_counts()  #total have 12,269 rows with types. Our dataset has 12,294 rows. Have 25 rows with null/blank types

TV         3787
OVA        3311
Movie      2348
Special    1676
ONA         659
Music       488
Name: type, dtype: int64

In [357]:
(df_raw['type'])[pd.isnull(df_raw['type'])]  = 'NaN' #Converts blank in 'type' to 'NaN'

C:\Users\miche\Documents\Anaconda\envs\fastai-cpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [358]:
df_raw['type'].value_counts()  #'type' column are now all filled, no more blanks

TV         3787
OVA        3311
Movie      2348
Special    1676
ONA         659
Music       488
NaN          25
Name: type, dtype: int64

In [307]:
df_raw #the full data

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [308]:
type(df_raw)  #this is a DataFrame

pandas.core.frame.DataFrame

In [309]:
df_raw=df_raw.iloc[:,:].values

In [310]:
df_raw #convert DataFrame to numpy array

array([[32281, 'Kimi no Na wa.', 'Drama, Romance, School, Supernatural',
        ..., '1', 9.37, 200630],
       [5114, 'Fullmetal Alchemist: Brotherhood',
        'Action, Adventure, Drama, Fantasy, Magic, Military, Shounen',
        ..., '64', 9.26, 793665],
       [28977, 'Gintama°',
        'Action, Comedy, Historical, Parody, Samurai, Sci-Fi, Shounen',
        ..., '51', 9.25, 114262],
       ...,
       [5621, 'Violence Gekiga David no Hoshi', 'Hentai', ..., '4', 4.88,
        219],
       [6133, 'Violence Gekiga Shin David no Hoshi: Inma Densetsu',
        'Hentai', ..., '1', 4.98, 175],
       [26081, 'Yasuji no Pornorama: Yacchimae!!', 'Hentai', ..., '1',
        5.46, 142]], dtype=object)

In [311]:
from sklearn.preprocessing import LabelEncoder

In [312]:
alltypes=LabelEncoder()

In [313]:
df_raw[:,3]

array(['Movie', 'TV', 'TV', ..., 'OVA', 'OVA', 'Movie'], dtype=object)

In [314]:
df_raw[:,3]=alltypes.fit_transform(df_raw[:,3])

In [315]:
df_raw

array([[32281, 'Kimi no Na wa.', 'Drama, Romance, School, Supernatural',
        ..., '1', 9.37, 200630],
       [5114, 'Fullmetal Alchemist: Brotherhood',
        'Action, Adventure, Drama, Fantasy, Magic, Military, Shounen',
        ..., '64', 9.26, 793665],
       [28977, 'Gintama°',
        'Action, Comedy, Historical, Parody, Samurai, Sci-Fi, Shounen',
        ..., '51', 9.25, 114262],
       ...,
       [5621, 'Violence Gekiga David no Hoshi', 'Hentai', ..., '4', 4.88,
        219],
       [6133, 'Violence Gekiga Shin David no Hoshi: Inma Densetsu',
        'Hentai', ..., '1', 4.98, 175],
       [26081, 'Yasuji no Pornorama: Yacchimae!!', 'Hentai', ..., '1',
        5.46, 142]], dtype=object)

In [316]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [328]:
ct=ColumnTransformer(transformers=[('encode',OneHotEncoder(),[3])],remainder='passthrough')

In [329]:
hi=ct.fit_transform(df_raw)

In [327]:
??ColumnTransformer

In [330]:
hi

array([[1.0, 0.0, 0.0, ..., '1', 9.37, 200630],
       [0.0, 0.0, 0.0, ..., '64', 9.26, 793665],
       [0.0, 0.0, 0.0, ..., '51', 9.25, 114262],
       ...,
       [0.0, 0.0, 0.0, ..., '4', 4.88, 219],
       [0.0, 0.0, 0.0, ..., '1', 4.98, 175],
       [1.0, 0.0, 0.0, ..., '1', 5.46, 142]], dtype=object)

In [331]:
hi=pd.DataFrame(hi)  #to convert back to DataFrame

In [336]:
hi

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,0,0,0,0,0,0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",1,9.37,200630
1,0,0,0,0,0,0,1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",64,9.26,793665
2,0,0,0,0,0,0,1,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",51,9.25,114262
3,0,0,0,0,0,0,1,9253,Steins;Gate,"Sci-Fi, Thriller",24,9.17,673572
4,0,0,0,0,0,0,1,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",51,9.16,151266
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0,0,0,0,1,0,0,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,1,4.15,211
12290,0,0,0,0,1,0,0,5543,Under World,Hentai,1,4.28,183
12291,0,0,0,0,1,0,0,5621,Violence Gekiga David no Hoshi,Hentai,4,4.88,219
12292,0,0,0,0,1,0,0,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,1,4.98,175


In [337]:
hi.rename(columns={'Movie','Music','NaN','ONA','OVA','Special','TV','anime_id','name','genre','episodes','rating','members'}, 
         inplace=True)

TypeError: 'set' object is not callable

In [338]:
Index([u'Movie',u'Music',u'NaN',u'ONA',u'OVA',u'Special',u'TV',u'anime_id',u'name',u'genre',u'episodes',u'rating',u'members'],
      dtype='object')

NameError: name 'Index' is not defined

In [340]:
hi.get_feature_names()

AttributeError: 'DataFrame' object has no attribute 'get_feature_names'

In [341]:
hi.head(2)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,0,0,0,0,0,0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",1,9.37,200630
1,0,0,0,0,0,0,1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",64,9.26,793665


In [ ]:
### alternative one hot encoding

In [344]:
df_raw

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [359]:
abc=pd.get_dummies(df_raw.type)

In [360]:
print(abc)  #tis is much faster.... some more have header names  # hav splited into diff types

       Movie  Music  NaN  ONA  OVA  Special  TV
0          1      0    0    0    0        0   0
1          0      0    0    0    0        0   1
2          0      0    0    0    0        0   1
3          0      0    0    0    0        0   1
4          0      0    0    0    0        0   1
...      ...    ...  ...  ...  ...      ...  ..
12289      0      0    0    0    1        0   0
12290      0      0    0    0    1        0   0
12291      0      0    0    0    1        0   0
12292      0      0    0    0    1        0   0
12293      1      0    0    0    0        0   0

[12294 rows x 7 columns]


In [376]:
## want to make separate columns from string #https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/

In [369]:
import pandas as pd

In [370]:
df_raw

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [385]:
newdf_raw=df_raw["genre"].str.split(",",n=6,expand=True)

In [386]:
newdf_raw  #this does not categorise into drama , romance , school, etc..

,0,1,2,3,4,5,6
0,Drama,Romance,School,Supernatural,None,None,None
1,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen
2,Action,Comedy,Historical,Parody,Samurai,Sci-Fi,Shounen
3,Sci-Fi,Thriller,None,None,None,None,None
4,Action,Comedy,Historical,Parody,Samurai,Sci-Fi,Shounen
...,...,...,...,...,...,...,...
12289,Hentai,None,None,None,None,None,None
12290,Hentai,None,None,None,None,None,None
12291,Hentai,None,None,None,None,None,None
12292,Hentai,None,None,None,None,None,None


In [403]:
efg=df_raw['genre'].str.get_dummies(sep=',')  #total have 82 genres

In [404]:
efg  #there are 62 rows that have blanks for genres, used excel sheet to see. Need to fill it up w "NaN"?

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,Shoujo,Shounen,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [399]:
type(efg)

pandas.core.frame.DataFrame

In [402]:
efg.sample(n=5)

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,Shoujo,Shounen,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi
9810,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11364,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [401]:
hi.sample(n=50)

,0,1,2,3,4,5,6,7,8,9,10,11,12
10913,0,0,0,0,0,0,1,31783,Yowamushi Pedal: New Generation,"Comedy, Drama, Shounen, Sports",Unknown,NaN,9037
9496,0,0,0,0,0,0,1,19873,Marude Dameo,"Comedy, Sci-Fi, Shounen",47,6.2,73
11800,0,0,0,0,1,0,0,3108,Miboujin: Numeriau Nikuyoku to Midara ni Nurer...,Hentai,2,6.07,1363
4147,0,0,0,0,0,0,1,3271,Bakuto Sengen Daigander,"Adventure, Mecha, Sci-Fi, Shounen",39,6.79,1174
4766,0,0,0,0,1,0,0,14145,"Hanayaka Nari, Waga Ichizoku: Kinetograph","Drama, Historical, Josei",2,6.64,8305
6056,0,0,0,0,0,0,1,21523,Baku Tech! Bakugan Gachi,"Action, Game, Shounen",39,6.26,690
1930,0,0,0,0,0,0,1,30694,Dragon Ball Super,"Action, Adventure, Comedy, Fantasy, Martial Ar...",Unknown,7.4,111443
3531,0,0,0,0,0,0,1,10346,Ring ni Kakero 1: Sekai Taikai-hen,Sports,6,6.96,566
8873,0,0,0,0,0,0,1,13207,Himitsukessha Taka no Tsume Gaiden: Mukashi no...,"Comedy, Historical, Parody",9,5.37,126
4672,0,0,0,0,0,0,1,3035,Chouon Senshi Borgman,"Action, Demons, Sci-Fi, Shounen",35,6.66,947


In [67]:
subset_rawless=df_raw.sample(n=1)

In [68]:
subset_rawless

,anime_id,name,genre,type,episodes,rating,members
8518,30026,Ekubo Ouji,"Comedy, Kids",TV,50.0,6.5,39


In [69]:
yless=subset_rawless['rating']

In [134]:
ygenre=subset_rawless['genre']

In [136]:
ygenre

8518    Comedy, Kids
Name: genre, dtype: object

In [143]:
def Convert(string):
    li = list(string.split(","))
    return li


In [189]:
str1="Comedy,Kids,Romance,Romance,Kids"


In [190]:
print(Convert(str1))

['Comedy', 'Kids', 'Romance', 'Romance', 'Kids']


In [191]:
from numpy import array


In [192]:
from numpy import argmax

In [193]:
from sklearn.preprocessing import LabelEncoder

In [194]:
from sklearn.preprocessing import OneHotEncoder

In [153]:
data=["hot","cold"]

In [195]:
values=array([((str1))])

In [196]:
values

array(['Comedy,Kids,Romance,Romance,Kids'], dtype='<U32')

In [198]:
print(values)

['Comedy,Kids,Romance,Romance,Kids']


In [197]:
label_encoder=LabelEncoder()

In [199]:
integer_encoded=label_encoder.fit_transform(values)


In [200]:
onehot_encoder=OneHotEncoder(sparse=False)

In [201]:
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)


In [202]:
print(integer_encoded)


[[0]]


In [204]:
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


In [203]:

print(onehot_encoded)


[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [205]:
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])


In [206]:
print(inverted)

['Comedy,Kids,Romance,Romance,Kids']


### Doing splitting

In [ ]:
# do we need to use 67% for training set , then 33% test set? 
# https://machinelearningmastery.com/how-to-prepare-categorical-data-for-deep-learning-in-python/

In [33]:
X_train, X_test, y_train,y_test= train_test_split(subset_raw.drop(['rating','name','anime_id','genre','type'], axis = 1),y, test_size=0.20, random_state = 0)

In [34]:
X_train.tail()

,episodes,members
6365,1.0,32917
901,2.0,11084
6960,1.0,224
4785,12.0,95168
5860,1.0,1470


In [ ]:
X_train.info()

From data - genre can be splited and converted to one hot encoding(but do with piping ??) <br>
splitting into franchise name? probably I can find another dataset with the franchise and name and do a join. <br>


In [ ]:
def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))

In [ ]:
set_rf_samples(5000)

In [ ]:
import dill

## alternative Code to save session 
else i just use pickle

In [ ]:
dill.dump_session('notebook_sessions.db')

#### To load

In [ ]:
dill.load_session('notebook_session.db')

## Graph plotting time(Visualisation)

In [ ]:
from ggplot import *

In [ ]:
??ggplot

Collinearity between rating and members - no observable relationship(?)

In [ ]:
ggplot(subset_df, aes(x = 'rating', y = 'members')) +\
geom_point(alpha = 0.5) +\
ylim(0,500) 

From the plot - 
1. None(world) has no events - maybe its like the start page or something
2. possible linear relationship between event count and game time? 
3. World seems to be independent of game_time and event_count? 

In [ ]:
import json

In [ ]:
subset_df.event_data.iloc[2]

### Converting the json format of event_data into more rows:
1. media_type
2. total_duration(?)

## Data cleaning/ dplyr time 
<list>
    <item>1. Franchises usually can be obtained by doing a string split on ":"(or its just the trend) -> create a new boolean variable (is_part_of_franchise) </item><br>
        <item>2. will have to split the genre by "," -> send through one hot encoding </item> <br>
        <item>3. Do simple imputing on the missing values for episode( maybe using mode?)</item>
</list>

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer

The **transformers(focus)** and predictors 

In [ ]:
comma_remover = FunctionTransformer(lambda types:types.str.extract('.+,.+') )

In [ ]:
num_transformer = Pipeline([
    ('num_imputer', SimpleImputer(strategy='constant', fill_value=30)),
    
])

type_transformer = Pipeline([('comma_remover', ])

## Building model

Basic Random forest using episodes, rating and members. 

In [ ]:
m = RandomForestRegressor(n_jobs=-1, random_state = 0)
m.fit(X_train,y_train)